In [9]:
"""
Given two strings s and t of lengths m and n respectively, return the minimum window substring of s such that every character in t (including duplicates) is included in the window. If there is no such substring, return the empty string "".

The testcases will be generated such that the answer is unique.

A substring is a contiguous sequence of characters within the string.

 

Example 1:

Input: s = "ADOBECODEBANC", t = "ABC"
Output: "BANC"
Explanation: The minimum window substring "BANC" includes 'A', 'B', and 'C' from string t.
Example 2:

Input: s = "a", t = "a"
Output: "a"
Explanation: The entire string s is the minimum window.
Example 3:

Input: s = "a", t = "aa"
Output: ""
Explanation: Both 'a's from t must be included in the window.
Since the largest window of s only has one 'a', return empty string.
 

Constraints:

m == s.length
n == t.length
1 <= m, n <= 105
s and t consist of uppercase and lowercase English letters.
 

Follow up: Could you find an algorithm that runs in O(m + n) time?

Tip:
    0. Top idea is that do a window traversal
        a. l = r = 0
        b. inc r until all of t is found; that's first l-r windows with desired chars.
        c. now decrement l and increment r for other such desired windows; and choose one with min r-l
    1. solution is through min-heap ---> O(nlogm)
        Select from k sorted list, atleast 1 from each list such that range is minimum
        a. Create a list of each char indices that should be in min-window str
            A = []
            B = []
            C = []
        b. Now as per the count, create an heap by adding elements from t.
            example A should be two times then we add two initial indices of A 
            from above list to heap
        c. Now once this is certain that we have heap with all chars required in
        sub-string
        d. Now we need to minimize the range of indices in heap
            keep a check on max_index
            and for every min idx in heap (at top) keep adding its next idx and check range value
        e. finally shorted range indices will be known.
    2. Counter is default dict, where it won't raise for key not added initially
    3. Another solution is that instead of min-heap ----> O(n + m)
        a. use deque
            why deque? if a char occurs more than 1 time hence deque
        b. prepare deque indices list for each char in t
        c. Once we have one complete t set
        d. then just iterate over remaining of s,
            keep updating deque[char] if char encoutered is in t, popleft and append-right; maintain a list of popped, which can be used to validate minIdx in case it is popped.
            keep updatinr start, end for result if range is min than previous
            need to prepare a deque for minIdx, can remove from mid of deque, so remove from top of minIdx if it is in popped.
"""


# Final try --- window traversal
from collections import Counter
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        target_count = len(t)
        target_chars = Counter(t)
        desired_chars = {}
        for char in t:
            desired_chars[char] = 0
        l, r   = 0, 0
        start, end = 0, float('inf')
        while r < len(s):
            if target_count > 0 and s[r] in t:
                desired_chars[s[r]] += 1
                if desired_chars[s[r]] <= target_chars[s[r]]:
                    target_count -= 1
            while target_count == 0:
                if r - l < end - start:
                    start, end = l, r
                charl = s[l]
                if charl in t:
                    desired_chars[charl] -= 1
                    if desired_chars[charl] < target_chars[charl]:
                        target_count += 1
                l += 1
            r += 1
        return "" if end == float('inf') else s[start:end+1]


# Try 1 --- Min Heap
from collections import defaultdict as dd, Counter
from heapq import heapify, heappush, heappop
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        char_map = dd(lambda: [])
        char_count = Counter(t)
        for idx, char in enumerate(s):
            if char in t:
                char_map[char].append(idx)
        elm_heap = []
        char_idx = {}
        for char in char_count:
            if len(char_map[char]) < char_count[char]:
                return ""
            else:
                elm_heap.extend(
                    [(char_map[char][idx], char) for idx in range(char_count[char])]
                )
                char_idx[char] = char_count[char]
        start, end = float('inf'), float('-inf')
        for elm, _ in elm_heap:
            start = min(elm, start)
            end   = max(elm, end)
        istart, iend = start, end
        heapify(elm_heap)
        while char_idx[elm_heap[0][1]] < len(char_map[elm_heap[0][1]]):
            curr_elm, curr_char = heappop(elm_heap)
            next_elm = char_map[curr_char][char_idx[curr_char]]
            char_idx[curr_char] += 1
            end = max(end, next_elm)
            heappush(elm_heap, (next_elm, curr_char))
            start = elm_heap[0][0]
            if end - start < iend - istart:
                iend, istart = end, start
        return s[istart:iend+1]

In [20]:
# Try 2 --- same idea as window traversal, though for min index we don't maintain l-->r thing rather we just keep popping and l-r is 
from collections import defaultdict as dd, Counter, deque
from heapq import heapify, heappush, heappop
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        char_count = Counter(t)
        char_idx   = dd(lambda: deque([]))
        idx = 0
        minIdx = deque([])
        popped = set()
        substr_l = 0
        while idx < len(s) and substr_l < len(t):
            curr_char = s[idx]
            if char_count[curr_char] > 0:
                minIdx.append(idx)
                char_idx[curr_char].append(idx)
                char_count[curr_char] -= 1
                substr_l += 1
            elif curr_char in t:
                minIdx.append(idx)
                popped.add(char_idx[curr_char].popleft())
                char_idx[curr_char].append(idx)
            idx += 1
        if substr_l < len(t):
            return ""
        while minIdx[0] in popped:
            minIdx.popleft()
        rstart, rend = minIdx[0], idx-1
        while idx < len(s):
            char = s[idx]
            if char in t:
                popped.add(char_idx[char].popleft())
                char_idx[char].append(idx)
                minIdx.append(idx)
                while minIdx[0] in popped:
                    minIdx.popleft()
                if idx - minIdx[0] < rend - rstart:
                    rstart, rend = minIdx[0], idx
            idx += 1
        return s[rstart:rend+1]